# normattiva2md - Quickstart

Questo notebook mostra come usare `normattiva2md` come libreria Python per convertire documenti legali italiani da normattiva.it a Markdown.

## Installazione

```bash
pip install normattiva2md
```

## 1. Conversione da URL

Il caso d'uso più comune: convertire un documento direttamente dall'URL di normattiva.it

In [ ]:
from normattiva2md import convert_url

# Legge Stanca (Legge 4/2004 - Accessibilità)
url = "https://www.normattiva.it/uri-res/N2Ls?urn:nir:stato:legge:2004-01-09;4"

result = convert_url(url, quiet=True)

if result:
    print(f"Titolo: {result.title}")
    print(f"Data GU: {result.data_gu}")
    print(f"Codice Redazionale: {result.codice_redaz}")
    print(f"\nLunghezza markdown: {len(result.markdown)} caratteri")
else:
    print("Conversione fallita")

### Visualizzare il contenuto

In [ ]:
# Mostra i primi 1500 caratteri
print(result.markdown[:1500])

### Salvare su file

In [ ]:
# Salva il markdown su file
result.save("legge_stanca.md")
print("File salvato: legge_stanca.md")

## 2. Estrarre un singolo articolo

Puoi estrarre solo un articolo specifico usando il parametro `article`:

In [ ]:
# Estrai solo l'articolo 3
result_art3 = convert_url(url, article="3", quiet=True)

if result_art3:
    print(result_art3.markdown)

### Articoli con estensione (bis, ter, etc.)

In [ ]:
# Esempio con articolo 16-bis (se presente nel documento)
# result_art16bis = convert_url(url, article="16bis", quiet=True)

## 3. Accedere ai metadata

L'oggetto `ConversionResult` contiene tutti i metadata del documento:

In [ ]:
# Accesso diretto ai metadata
print("Metadata completi:")
for key, value in result.metadata.items():
    print(f"  {key}: {value}")

In [ ]:
# Shortcut properties
print(f"Data GU: {result.data_gu}")
print(f"Codice Redaz: {result.codice_redaz}")
print(f"Data Vigenza: {result.data_vigenza}")
print(f"URL: {result.url}")
print(f"URL XML: {result.url_xml}")

## 4. Generare link ai riferimenti

Con `with_urls=True`, i riferimenti normativi nel testo diventano link cliccabili:

In [ ]:
result_with_links = convert_url(url, with_urls=True, quiet=True)

# Cerca link nel markdown
import re
links = re.findall(r'\[([^\]]+)\]\(https://www\.normattiva\.it[^)]+\)', result_with_links.markdown)
print(f"Trovati {len(links)} link a normattiva.it")
for link in links[:5]:
    print(f"  - {link}")

## 5. Classe Converter per uso avanzato

Per batch processing o configurazione persistente, usa la classe `Converter`:

In [ ]:
from normattiva2md import Converter

# Crea converter con configurazione
conv = Converter(quiet=True)

# Batch processing
urls = [
    "https://www.normattiva.it/uri-res/N2Ls?urn:nir:stato:legge:2004-01-09;4",
    "https://www.normattiva.it/uri-res/N2Ls?urn:nir:stato:decreto.legislativo:2005-03-07;82",
]

results = []
for url in urls:
    result = conv.convert_url(url)
    if result:
        results.append(result)
        print(f"OK: {result.title[:50]}...")

print(f"\nConvertiti {len(results)}/{len(urls)} documenti")

## 6. Ricerca in linguaggio naturale

Con una API key Exa AI, puoi cercare documenti in linguaggio naturale:

```python
from normattiva2md import search_law

# Richiede: export EXA_API_KEY='your-key'
results = search_law("legge stanca accessibilità")
for r in results:
    print(f"[{r.score:.2f}] {r.title}")
    print(f"  URL: {r.url}")
```

### Ricerca e conversione in un passo

In [ ]:
# Richiede EXA_API_KEY configurata
# conv = Converter(exa_api_key="your-key", quiet=True)
# result = conv.search_and_convert("decreto dignità")
# result.save("decreto_dignita.md")

## 7. Gestione errori

In [ ]:
from normattiva2md import (
    convert_url,
    InvalidURLError,
    ConversionError,
    Normattiva2MDError,
)

# Test URL invalido
try:
    result = convert_url("https://example.com/not-normattiva")
except InvalidURLError as e:
    print(f"URL non valido: {e}")
except Normattiva2MDError as e:
    print(f"Errore generico: {e}")

In [ ]:
# Articolo non trovato (errore soft - ritorna None)
result = convert_url(url, article="999", quiet=True)
if result is None:
    print("Articolo 999 non trovato nel documento")

## 8. Uso con pandas (esempio)

In [ ]:
# Esempio: creare un DataFrame con i metadata di più documenti
import pandas as pd

data = []
for r in results:
    data.append({
        "titolo": r.title,
        "data_gu": r.data_gu,
        "codice_redaz": r.codice_redaz,
        "lunghezza": len(r.markdown),
    })

df = pd.DataFrame(data)
df

## Risorse

- **Repository**: https://github.com/ondata/normattiva_2_md
- **PyPI**: https://pypi.org/project/normattiva2md/
- **Documentazione URL**: vedere `docs/URL_NORMATTIVA.md` nel repository